<h1 align="center"> Clustering </h1>

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

import matplotlib as mpl
mpl.rc("savefig", dpi=100) # Adjust for higher-resolution figures

<h2> Lloyd's algorithm from scratch </h2>

<h3> Initializing the <i>k</i> centers </h3>

In [6]:
def init_centers(X, k):
    """
    Randomly samples k observations from X as centers.
    Returns these centers as a (k x d) numpy array.
    
    (X,k) --> centers(numpy.array)
    """
    #
    centers = np.random.choice(len(X), size=k, replace= False)
    return X[centers , :]
    #

In [22]:
#demo run
k=2
X=np.array([ [1,2],[2,3],[1,1],[2,4],[1,2],[1,5],[7,1]])
centers= init_centers(X,k)
centers


In [26]:
k2=3
X2=np.array([[1,1,2],[1,1,9],[2,2,0],[3,2,1],[2,2,1],[2,2,4],[1,1,3]])
centers2=init_centers(X2,k2)

<p><span style="color:purple">define a function "initC" from scratch</span>

In [9]:
#def initC(X,k):
#   uncomment to check
#Centers=initC(X,k);Centers

array([[1, 5],
       [2, 3]])

<h3>2. Computing the distances</h3>

In [40]:
def compute_d2(X, centers):

    #(X, centers --> S (distance matrix))   

    m = len(X)
    k = len(centers)
    
    S = np.empty((m, k))
    for i in range (m): #for each observation, item
        S[i,:] = np.linalg.norm(X[i, : ] - centers , ord=2, axis=1)**2 
        # 	2-norm (largest sing. value)
    return S

In [19]:
#demo run
S=compute_d2(X, centers)
S

array([[37.,  5.],
       [29.,  1.],
       [36., 10.],
       [34.,  0.],
       [37.,  5.],
       [52.,  2.],
       [ 0., 34.]])

In [27]:
S2=compute_d2(X2,centers2)
S2

array([[ 6.,  6., 49.],
       [27., 83.,  0.],
       [16.,  0., 83.],
       [10.,  2., 69.],
       [ 9.,  1., 66.],
       [ 0., 16., 27.],
       [ 3., 11., 36.]])

In [41]:
a=np.array([[2,3,4]])
b=np.array([[6,6,6]])
aa=np.array([2,3,4])
bb=np.array([6,6,6])


`numpy.linalg.norm(array1 - array2 , ord=2, axis=1) **2` <br>
https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.linalg.norm.html

<p><span style="color:purple">compute distance between vectors a and b</span>

In [45]:
normab = np.linalg.norm(a - b, ord=2, axis=1 )**2
print(normab)
normabb = np.linalg.norm(aa - bb, ord=2, axis=0 )**2
normabb

[29.]


28.999999999999996

<h3>3. Assign cluster labels </h3>
<p>(to distance matrix 'S')

In [19]:
def assign_cluster_labels(S):
'''
S --> labels
'''
    return np.argmin(S, axis=1) ## 1 ; the column index = [1]
    #

In [45]:
#demo run
labels = assign_cluster_labels(S)
labels

array([0, 0, 0, 1, 0, 1, 0])

<h3>4. Compute the centers of each cluster </h3>

In [21]:
def update_centers(X, y):
    '''
    (X , labels) --> centers
    '''
    # X[:m, :d] == m points, each of dimension d
    # y[:m] == cluster labels
    m, d = X.shape
    k = max(y) + 1
    assert m == len(y)
    assert (min(y) >= 0)
    
    centers = np.empty((k, d))
    for j in range(k):
        # Compute the new center of cluster j,
        # i.e., centers[j, :d].
        #
        centers[j,:] = np.mean( X[y==j, : ], axis = 0)
        #
    return centers

In [42]:

centers = update_centers(X, labels)
centers

array([[7.        , 1.        ],
       [1.33333333, 2.83333333]])

<h4>HELPERS: return the within-cluster sum of squares, check convergence </h4>

In [22]:
def WCSS(S):
    #
    return np.sum( np.amin(S, axis = 1))
    #
    


In [23]:
def has_converged(old_centers, centers):
    return set([tuple(x) for x in old_centers]) == set([tuple(x) for x in centers])

<h3> The main `kmeans` function </h3>

In [24]:
df = pd.read_csv('data.csv')
df.head()

Unnamed: 0       x_1       x_2  label
0           0 -0.234443 -1.075960      1
1           1  0.730359 -0.918093      0
2           2  1.432270 -0.439449      0
3           3  0.026733  1.050300      0
4           4  1.879650  0.207743      0

In [25]:
points = df[['x_1', 'x_2']].values
labels = df['label'].values
n, d = points.shape
k = 2

In [26]:
def kmeans(X, k,
           starting_centers=None,
           max_steps=np.inf):
    # 1. set centers
    if starting_centers is None:
        centers = init_centers(X, k)
    else:
        centers = starting_centers
        
    converged = False # whether the centers have "moved"
    labels = np.zeros(len(X))
    i = 1
    while (not converged) and (i <= max_steps):
        old_centers = centers
        #
        #2 _find square dist points-centers        STEP 1 in Lloyds
        S = compute_d2(X, centers) #gets distance matrix
        
        #3 assign the points to a cluster
        labels = assign_cluster_labels(S)
        
        #4__recalculate the centroids to a center   STEP 2 in Lloyd's
        centers = update_centers(X, labels)
        
        #5__check if centroids have moved
        converged = has_converged(old_centers, centers)
        #
        print ("iteration", i, "WCSS = ", WCSS (S))
        i += 1
    return labels

clustering = kmeans(points, k, starting_centers=points[[0, 187], :])

iteration 1 WCSS =  549.9175535488309
iteration 2 WCSS =  339.80066330255096
iteration 3 WCSS =  300.330112922328
iteration 4 WCSS =  289.80700777322045
iteration 5 WCSS =  286.0745591062787
iteration 6 WCSS =  284.1907705579879
iteration 7 WCSS =  283.22732249939105
iteration 8 WCSS =  282.456491302569
iteration 9 WCSS =  281.84838225337074
iteration 10 WCSS =  281.57242082723724
iteration 11 WCSS =  281.5315627987326


In [27]:
clustering = kmeans(points, k)

iteration 1 WCSS =  342.42571680019324
iteration 2 WCSS =  289.9323284147895
iteration 3 WCSS =  286.7538516279051
iteration 4 WCSS =  284.09771994025346
iteration 5 WCSS =  283.36215702221284
iteration 6 WCSS =  282.54826291334257
iteration 7 WCSS =  282.10232370657434
iteration 8 WCSS =  281.57242082723724
iteration 9 WCSS =  281.5315627987326
